In [1]:
##import packages

import requests
import json
import pandas as pd
import numpy as np
from decimal import Decimal
import itertools as it
import plotly.express as px


In [ ]:
##Request fixtures from FPL API

url = f"https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
json_data = response.json()

Fixtures_Dim = json_data

Fixtures_Dim = pd.json_normalize(Fixtures_Dim)


Fixtures_Dim_Future=Fixtures_Dim[Fixtures_Dim['finished']==False]

### Used as variable to return only the data for completed gameweeks

UpcomingWeek = Fixtures_Dim_Future['event'].min().astype(int)
PrevWeek = UpcomingWeek-1

### Variable for number of gameweeks to go back to calculate averages from

NoTrailingWeeks = 5

NoForecastWeeks = 4

TeamID = 1769272

print(UpcomingWeek)
print(PrevWeek)


22
21


In [3]:
###Request Team Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

teams_dim = json_data['teams']

Teams_Dim = pd.json_normalize(teams_dim)



In [4]:
###Duplicates of Fixtures for Home and Away, so that this can be merged with elements

Fixtures_Away1 = Fixtures_Dim_Future
Fixtures_Away1.head()

Fixtures_Home1 = Fixtures_Dim_Future
Fixtures_Home1.head()

FixturesConcat = pd.concat([Fixtures_Away1,Fixtures_Home1], keys=["Away", "Home"])

FixturesConcat.reset_index(inplace=True)

FixturesConcat['TeamID'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Away' else row['team_h'], 
    axis=1
)

FixturesConcat['OppTeam2'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Home' else row['team_h'], 
    axis=1
)

In [5]:
### Gameweek elements data from FPL API

player_data = []

for gw1 in range(1,UpcomingWeek):
    url = f"https://fantasy.premierleague.com/api/event/{gw1}/live/"
    
    response = requests.get(url)
    json_data = response.json()
    print(url)

    for gw in json_data['elements']:
        #i1 = gw['id']
        playerid=gw['id']
        minutes=gw['stats']['minutes']
        goals_scored=gw['stats']['goals_scored']
        assists=gw['stats']['assists']
        clean_sheets=gw['stats']['clean_sheets']
        goals_conceded=gw['stats']['goals_conceded']
        own_goals=gw['stats']['own_goals']
        penalties_saved=gw['stats']['penalties_saved']
        penalties_missed=gw['stats']['penalties_missed']
        yellow_cards=gw['stats']['yellow_cards']
        red_cards=gw['stats']['red_cards']
        saves=gw['stats']['saves']
        bonus=gw['stats']['bonus']
        bps=gw['stats']['bps']
        influence=gw['stats']['influence']
        creativity=gw['stats']['creativity']
        threat=gw['stats']['threat']
        ict_index=gw['stats']['ict_index']
        starts=gw['stats']['starts']
        expected_goals=gw['stats']['expected_goals']
        expected_assists=gw['stats']['expected_assists']
        expected_goal_involvements=gw['stats']['expected_goal_involvements']
        expected_goals_conceded=gw['stats']['expected_goals_conceded']
        total_points=gw['stats']['total_points']
        if gw['explain']: 
            fixtureid = gw['explain'][0]['fixture'] 
        else: 
            fixtureid = None

        player_data.append({
            'playerid': playerid,
            'minutes': minutes,
            'goals_scored': goals_scored,
            'assists': assists,
            'clean_sheets': clean_sheets,
            'goals_conceded': goals_conceded,
            'threat': threat,
            'expected_goals': expected_goals,
            'expected_assists': expected_assists,
            'expected_goal_involvements': expected_goal_involvements,
            'expected_goals_conceded': expected_goals_conceded,
            'fixtureid': fixtureid  


            })

    GWElementsDF = pd.DataFrame(player_data)
GWElementsDF.head(5)

GWElementsDF1 = pd.merge(GWElementsDF, Fixtures_Dim,left_on='fixtureid', right_on='id', how='outer')

GWElementsDF1.to_csv('FPL_Event_By_Week.csv')
GWElementsDF1.head()


https://fantasy.premierleague.com/api/event/1/live/
https://fantasy.premierleague.com/api/event/2/live/
https://fantasy.premierleague.com/api/event/3/live/
https://fantasy.premierleague.com/api/event/4/live/
https://fantasy.premierleague.com/api/event/5/live/
https://fantasy.premierleague.com/api/event/6/live/
https://fantasy.premierleague.com/api/event/7/live/
https://fantasy.premierleague.com/api/event/8/live/
https://fantasy.premierleague.com/api/event/9/live/
https://fantasy.premierleague.com/api/event/10/live/
https://fantasy.premierleague.com/api/event/11/live/
https://fantasy.premierleague.com/api/event/12/live/
https://fantasy.premierleague.com/api/event/13/live/
https://fantasy.premierleague.com/api/event/14/live/
https://fantasy.premierleague.com/api/event/15/live/
https://fantasy.premierleague.com/api/event/16/live/
https://fantasy.premierleague.com/api/event/17/live/
https://fantasy.premierleague.com/api/event/18/live/
https://fantasy.premierleague.com/api/event/19/live/
ht

,playerid,minutes_x,goals_scored,assists,clean_sheets,goals_conceded,threat,expected_goals,expected_assists,expected_goal_involvements,...,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
1,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
2,3.0,90.0,0.0,0.0,1.0,0.0,8.0,0.12,0.02,0.14,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
3,4.0,90.0,1.0,1.0,1.0,0.0,46.0,0.45,0.04,0.49,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0


In [6]:
###Standard Element Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

elements_dim = json_data['elements']

Elements_Dim1 = pd.json_normalize(elements_dim)

Elements_Dim1.to_csv('elementsdim.csv')

Elements_Dim1 = pd.merge(GWElementsDF1, Elements_Dim1,left_on='playerid', right_on='id', how='outer',suffixes=('.gw', '.info'))

Elements_Dim1['ExcludeDef'] = Elements_Dim1.apply(
    lambda row: 'Y' if (row['element_type'] == '1' or row['element_type'] == '2') and row['minutes_x'] > 59 else 'N', 
    axis=1
)

Elements_Dim1.to_csv('elements4wk.csv')

In [9]:
###Arbitrary team weightings calculated based on average of last season's odds

TeamWeightings = {
    "Weighting": [
        1.699153895, 1.68275948, 1.639154108, 1.626396476, 1.605876872, 1.542304678,
        1.526424188, 1.50219836, 1.499125082, 1.489893392, 1.44492154, 1.424635514,
        1.365324148, 1.288771283, 1.266010528, 1.251813146, 1.18226904, 1.7, 1.7, 1.7
    ],
    "Teams.id": [
        3, 7, 16, 20, 4, 9, 8, 2, 15, 19, 5, 14, 6, 12, 18, 1, 13, 10, 11, 17
    ],
    "Teams.name": [
        "Bournemouth", "Crystal Palace", "Nott'm Forest", "Wolves", "Brentford", "Fulham",
        "Everton", "Aston Villa", "Newcastle", "West Ham", "Brighton", "Man Utd", "Chelsea",
        "Liverpool", "Spurs", "Arsenal", "Man City", "Ipswich", "Leicester", "Southampton"
    ]
}

TeamWeightings = pd.DataFrame(TeamWeightings)

Elements_Dim1['OppTeam'] = Elements_Dim1.apply(lambda row: row['team_h'] if row['team_h'] == row['team'] else row['team_a'], axis=1)

Elements_Dim2 = pd.merge(Elements_Dim1, TeamWeightings,left_on='OppTeam', right_on='Teams.id', how='outer')

Elements_Dim2['XGC_Weighted'] = Elements_Dim2['Weighting'].astype(float)*Elements_Dim2['expected_goals_conceded.gw'].astype(float)

Elements_Dim2['XGI_Weighted'] = Elements_Dim2['expected_goal_involvements.gw'].astype(float)/Elements_Dim2['Weighting'].astype(float)



In [ ]:
###Filtered to only include the previous 4 weeks' data before it is grouped to return total sum for each team 

Elements_Dim2Filtered = Elements_Dim2[Elements_Dim2['event']>PrevWeek-NoTrailingWeeks]

Elements_Dim2Grouped1 = Elements_Dim2Filtered.groupby(['OppTeam','Teams.name', 'event']).agg(
    XGI_Weighted=('XGI_Weighted', 'sum'),
    XGC_Weighted=('XGC_Weighted', 'max'),
    Minutes=('minutes_x', 'sum')
).reset_index()



##Average of the above taken for each team
Elements_Dim2Grouped2 = Elements_Dim2Grouped1.groupby(['OppTeam','Teams.name']).agg(
    XGI_Weighted=('XGI_Weighted', 'mean'),
    XGC_Weighted=('XGC_Weighted', 'mean'),
    Minutes=('Minutes', 'mean')
).reset_index()



Elements_Dim2Grouped2.head()

Elements_Dim2Grouped2.to_csv('TeamProfiles.csv')

In [22]:
##Previous 4 Gameweeks this dataframe is returning the averages for the selected players
Elements_Dim1 = Elements_Dim1[Elements_Dim1['event']<UpcomingWeek]

GWElementsDFLast4Weeks = Elements_Dim1[Elements_Dim1['event']>UpcomingWeek-NoTrailingWeeks-1]

GWElementsDFLast4Weeks = GWElementsDFLast4Weeks[['minutes_x','playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']]

GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goal_involvements.gw'] = GWElementsDFLast4Weeks['expected_goal_involvements.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goals_conceded.gw'] = GWElementsDFLast4Weeks['expected_goals_conceded.gw'].astype(float)

GWElementsDFLast4Weeks = GWElementsDFLast4Weeks.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['minutes_x','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']].mean(numeric_only=True)

GWElementsDFLast4Weeks['ExcludeDef'] = GWElementsDFLast4Weeks.apply(
    lambda row: 'Y' if (row['element_type'] == 1 or row['element_type'] == 2) and row['minutes_x']< 59 else 'N', 
    axis=1
)

grouped_dfLast4Weeks=GWElementsDFLast4Weeks[GWElementsDFLast4Weeks['ExcludeDef']!='Y']
grouped_dfLast4Weeks.head()

,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.00,0.0000,0.0000,0.0000,0.0000,N
1,2.0,G.Jesus,4.0,68.0,1.0,48.25,0.2425,0.0325,0.2750,0.3925,N
2,3.0,Gabriel,2.0,63.0,1.0,67.50,0.1650,0.0250,0.1900,0.4600,N
3,4.0,Havertz,4.0,79.0,1.0,77.50,0.5900,0.0475,0.6375,0.4850,N
5,6.0,J.Timber,2.0,57.0,1.0,88.75,0.0100,0.1000,0.1100,0.4900,N


In [23]:
### Returns combinations of historic XG stats from players multiplied by historic opposition team performance

OpportunitiesCalc = pd.merge(grouped_dfLast4Weeks, Elements_Dim2Grouped2,left_on='team', right_on='OppTeam', how='outer')

OpportunitiesCalc['AttackOpp2'] = OpportunitiesCalc['XGC_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc['DefenceOpp2'] = OpportunitiesCalc['XGI_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc.sort_values(by=['AttackOpp2'])


,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef,OppTeam,XGI_Weighted,XGC_Weighted,Minutes,AttackOpp2,DefenceOpp2
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000000,0.0000,0.000000,0.000000,0.000000,N,1.0,0.783435,0.625907,281.714286,0.000000,0.000000
480,682.0,Edozie,3.0,45.0,20.0,0.000000,0.0000,0.000000,0.000000,0.000000,N,20.0,0.323239,1.406833,282.000000,0.000000,0.000000
18,638.0,Oulad M'hand,3.0,45.0,1.0,0.000000,0.0000,0.000000,0.000000,0.000000,N,1.0,0.783435,0.625907,281.714286,0.000000,0.000000
19,659.0,Kabia,3.0,45.0,1.0,0.000000,0.0000,0.000000,0.000000,0.000000,N,1.0,0.783435,0.625907,281.714286,0.000000,0.000000
20,665.0,Butler-Oyedeji,4.0,45.0,1.0,0.000000,0.0000,0.000000,0.000000,0.000000,N,1.0,0.783435,0.625907,281.714286,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,306.0,Vardy,4.0,56.0,11.0,59.500000,0.3050,0.100000,0.405000,1.470000,N,11.0,0.213025,3.527500,281.857143,1.428637,0.086275
282,328.0,M.Salah,3.0,135.0,12.0,88.666667,0.9200,0.306667,1.226667,0.893333,N,12.0,0.940669,1.207149,221.923077,1.480770,1.153888
404,82.0,Solanke,4.0,75.0,18.0,87.750000,0.4525,0.060000,0.512500,2.345000,N,18.0,0.647704,2.971960,281.642857,1.523129,0.331948
423,503.0,Son,3.0,98.0,18.0,74.000000,0.4200,0.235000,0.655000,2.177500,N,18.0,0.647704,2.971960,281.642857,1.946634,0.424246


In [24]:
##### Returns Output 1 FixturesElementsAll
FixturesElementsAll1 = pd.merge(FixturesConcat, Elements_Dim2Grouped2,left_on='OppTeam2', right_on='OppTeam', how='outer')

FixturesElementsAll2 = pd.merge(grouped_dfLast4Weeks, FixturesElementsAll1,left_on='team', right_on='TeamID', how='outer')

FixturesElementsAll2['AttackOpp_pergame'] = FixturesElementsAll2['XGC_Weighted']*FixturesElementsAll2['expected_goal_involvements.gw']

FixturesElementsAll2['DefenceOpp_pergame'] = FixturesElementsAll2['XGI_Weighted']*FixturesElementsAll2['expected_goals_conceded.gw']

FixturesElementsAll2.to_csv('FixturesElementsAll.csv')

fig = px.line(FixturesElementsAll2, x='event', y='AttackOpp_pergame', color='web_name', title='AttackOpp_pergame by Event')

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [12]:
##### Returns Output 2 FixturesElementsAllGrouped for previous 4 weeks

FixturesElementsAllGrouped = FixturesElementsAll2[FixturesElementsAll2['event']>UpcomingWeek-NoForecastWeeks-1]

FixturesElementsAllGrouped = FixturesElementsAllGrouped[['playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']]

FixturesElementsAllGrouped = FixturesElementsAllGrouped.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']].mean(numeric_only=True)

FixturesElementsAllGrouped.to_csv('FixturesElementsAllGrouped.csv')

In [13]:
url = f"https://fantasy.premierleague.com/api/entry/{TeamID}/event/{PrevWeek}/picks/"
response = requests.get(url)
json_data = response.json()

TeamSelection = json_data['picks']

TeamSelection = pd.json_normalize(TeamSelection)

TeamSelection.head()

,element,position,multiplier,is_captain,is_vice_captain,element_type
0,15,1,1,False,False,1
1,594,2,1,False,False,2
2,573,3,1,False,False,2
3,422,4,1,False,False,2
4,366,5,1,False,False,3


In [14]:
FixturesElementsAllGroupedwithTS = pd.merge(FixturesElementsAllGrouped, TeamSelection,left_on='playerid', right_on='element', how='outer')

FixturesElementsAllGroupedwithTS['IsInCurrentTeam'] = np.where(
    FixturesElementsAllGroupedwithTS['is_captain'].notnull(),
    'Y',
    'N'
)

In [15]:
FixturesElementsAllGroupedwithTS.head()

,playerid,web_name,element_type_x,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,XGI_Weighted,XGC_Weighted,AttackOpp_pergame,DefenceOpp_pergame,element,position,multiplier,is_captain,is_vice_captain,element_type_y,IsInCurrentTeam
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.0000,0.0000,0.0000,0.0000,0.457865,2.254207,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N
1,2.0,G.Jesus,4.0,68.0,1.0,0.2425,0.0325,0.2750,0.3925,0.457865,2.254207,0.619907,0.179712,NaN,NaN,NaN,NaN,NaN,NaN,N
2,3.0,Gabriel,2.0,63.0,1.0,0.1650,0.0250,0.1900,0.4600,0.457865,2.254207,0.428299,0.210618,NaN,NaN,NaN,NaN,NaN,NaN,N
3,4.0,Havertz,4.0,79.0,1.0,0.5900,0.0475,0.6375,0.4850,0.457865,2.254207,1.437057,0.222065,NaN,NaN,NaN,NaN,NaN,NaN,N
4,6.0,J.Timber,2.0,57.0,1.0,0.0100,0.1000,0.1100,0.4900,0.457865,2.254207,0.247963,0.224354,NaN,NaN,NaN,NaN,NaN,NaN,N


In [16]:
FixturesElementsAllGroupedwithTS.to_csv('FixturesElementsAllGroupedwithTS.csv')

In [17]:
##Import the packages to carry out the optimisation problem

import requests
from sklearn.preprocessing import MinMaxScaler
import warnings
from pulp import *
Small_Elements_df = FixturesElementsAllGroupedwithTS

ModuleNotFoundError: No module named 'sklearn'

In [18]:
def AttackOptimisationproblem(Source,AttSpend,FreeTransfers):
    Attack_Elements_df = Source.dropna(subset=['AttackOpp_pergame'])
    Attack_Elements_df['AttackOpp_pergame'].astype(float)
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=1]
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=2]
    ## Attack Variables
    Players = list(Attack_Elements_df['web_name'])
    Cost = dict(zip(Players, Attack_Elements_df['now_cost']))
    Score = dict(zip(Players, Attack_Elements_df['AttackOpp_pergame'].astype(float)))
    positions = dict(zip(Players, Attack_Elements_df['element_type_x']))
    CurrentSelection = dict(zip(Players, Attack_Elements_df['IsInCurrentTeam']))
    
    player_vars = LpVariable.dicts("Player", Players, lowBound=0, upBound=1, cat='Integer')

    ##Attack Constraints and problem solve

    total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)
    total_score += lpSum([Score[i] * player_vars[i] for i in player_vars])
    total_score += lpSum([Cost[i] * player_vars[i] for i in player_vars]) <= AttSpend

    MF = [p for p in positions.keys() if positions[p] == 3]
    ST = [p for p in positions.keys() if positions[p] == 4]
    CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']
    total_score += lpSum([player_vars[i] for i in MF]) == 5
    total_score += lpSum([player_vars[i] for i in ST]) == 3
    total_score += lpSum([player_vars[i] for i in CS]) == 8-FreeTransfers

    total_score.solve()

    for v in total_score.variables():
        if v.varValue > 0:
            print(v.name)


In [19]:
def DefenceOptimisationproblem(Source,DefSpend,FreeTransfers):
    Def_Elements_df=Source.dropna(subset=['DefenceOpp_pergame'])
    Def_Elements_df['DefenceOpp_pergame'].astype(float)
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=3]
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=4]
    ## Defence Variables
    DefPlayers = list(Def_Elements_df['web_name'])
    DefCost = dict(zip(DefPlayers, Def_Elements_df['now_cost']))
    DefScore = dict(zip(DefPlayers, Def_Elements_df['DefenceOpp_pergame'].astype(float)))
    Defpositions = dict(zip(DefPlayers, Def_Elements_df['element_type_x']))
    DefCurrentSelection = dict(zip(DefPlayers, Def_Elements_df['IsInCurrentTeam']))

    player_vars_def = LpVariable.dicts("DefPlayers", DefPlayers, lowBound=0, upBound=1, cat='Integer')

##Defence Constraints and problem solve

    total_score_def = LpProblem("Fantasy_Points_Problem_Defence", LpMinimize)
    total_score_def += lpSum([DefScore[i] * player_vars_def[i] for i in player_vars_def])
    total_score_def += lpSum([DefCost[i] * player_vars_def[i] for i in player_vars_def]) <= DefSpend

    GK = [p for p in Defpositions.keys() if Defpositions[p] == 1]
    DF = [p for p in Defpositions.keys() if Defpositions[p] == 2]
    #CS = [p for p in DefCurrentSelection.keys() if DefCurrentSelection[p] == 'N']
    total_score_def += lpSum([player_vars_def[i] for i in GK]) == 2
    total_score_def += lpSum([player_vars_def[i] for i in DF]) == 5
    #total_score_def += lpSum([player_vars_def[i] for i in CS]) == 7-FreeTransfers

    total_score_def.solve()

    for v in total_score_def.variables():
        if v.varValue > 0:
            print(v.name)

In [20]:
AttackOptimisationproblem(Small_Elements_df,700,1)
DefenceOptimisationproblem(Small_Elements_df,300,1)

Player_B.Fernandes
Player_Bowen
Player_Duran
Player_Gordon
Player_Isak
Player_Palmer
Player_Szoboszlai
Player_Vardy
DefPlayers_Chalobah
DefPlayers_Doherty
DefPlayers_Fabianski
DefPlayers_Henderson
DefPlayers_N.Williams
DefPlayers_Tosin
DefPlayers_Toti
